# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset
from azureml.core.webservice import Webservice
from azureml.core.webservice import AciWebservice
from azureml.core.model import InferenceConfig
from azureml.core.model import Model

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.26.0


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()
ws.write_config(path='.azureml')
# Name of the experiment
experiment_name = 'automl'
experiment=Experiment(ws, experiment_name)

# Get the data of Kaggle Titanic Dataset
key = "titanic-dataset"
found = False

if key in ws.datasets.keys(): 
    found = True
    dataset = ws.datasets[key] 

if not found:
    # Create AML Dataset and register it into Workspace
    titanicData = 'https://raw.githubusercontent.com/kkech/UdacityProj3_MS/master/train.csv'
    dataset = Dataset.Tabular.from_delimited_files(titanicData)
    #Register Dataset in Workspace
    dataset = dataset.register(workspace=ws,
                               name=key)


df = dataset.to_pandas_dataframe()
df.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [ ]:
## AnlCompute Cluster

In [8]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
compute_cluster_name = "titaniclusters"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=compute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS12_V2',
                                                           max_nodes=10)
    compute_target = ComputeTarget.create(ws, compute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)


Creating...
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [10]:
# TODO: Put your automl settings here
automl_settings = {
                    "max_concurrent_iterations": 9,
                    "primary_metric" : 'accuracy'
}
project_folder = './capstone-project'

# TODO: Put your automl config here
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             experiment_timeout_minutes = 20,
                             training_data=dataset,
                             label_column_name="Survived",   
                             path = project_folder,
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings)

In [11]:
remote_run = experiment.submit(automl_config, show_output=True)

Submitting remote run.
No run_configuration provided, running on titaniclusters with default configuration
Running on remote compute: titaniclusters


Experiment,Id,Type,Status,Details Page,Docs Page
automl,AutoML_2e98f987-8c25-4dd3-a149-4ea7f8c9b5c1,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Cross validation
STATUS:       DONE
DESCRIPTION:  Each iteration of the trained model was validated through cross-validation.
              
DETAILS:      
+---------------------------------+
|Number of folds                  |
+=================================+
|10                               |
+---------------------------------+

****************************************************************************************************

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: 

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [12]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show()
remote_run.wait_for_completion(show_output=True)

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

Experiment,Id,Type,Status,Details Page,Docs Page
automl,AutoML_2e98f987-8c25-4dd3-a149-4ea7f8c9b5c1,automl,Completed,Link to Azure Machine Learning studio,Link to Documentation




****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Cross validation
STATUS:       DONE
DESCRIPTION:  Each iteration of the trained model was validated through cross-validation.
              
DETAILS:      
+---------------------------------+
|Number of folds                  |
+=================================+
|10                               |
+---------------------------------+

****************************************************************************************************

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       DONE
DESCRIPTION:

{'runId': 'AutoML_2e98f987-8c25-4dd3-a149-4ea7f8c9b5c1',
 'target': 'titaniclusters',
 'status': 'Completed',
 'startTimeUtc': '2021-04-19T20:09:56.051852Z',
 'endTimeUtc': '2021-04-19T20:28:29.921733Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': None,
  'target': 'titaniclusters',
  'AMLSettingsJsonString': '{"path":null,"name":"automl","subscription_id":"510b94ba-e453-4417-988b-fbdc37b55ca7","resource_group":"aml-quickstarts-143180","workspace_name":"quick-starts-ws-143180","region":"southcentralus","compute_target":"titaniclusters","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"enable_batch_run":false,"enable_run_restructure":false,"iterations":1000,"primary_metric":"accuracy","task_type":"classification","data_script":null,"test_size":0.0,"validation_size":

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [13]:
best_run_automl, fitted_model_automl = remote_run.get_output()
model_name = best_run_automl.properties['model_name']

In [14]:
print(best_run_automl)

Run(Experiment: automl,
Id: AutoML_2e98f987-8c25-4dd3-a149-4ea7f8c9b5c1_46,
Type: azureml.scriptrun,
Status: Completed)


In [15]:
fitted_model_automl.steps

[('datatransformer',
  DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                  feature_sweeping_config=None, feature_sweeping_timeout=None,
                  featurization_config=None, force_text_dnn=None,
                  is_cross_validation=None, is_onnx_compatible=None, logger=None,
                  observer=None, task=None, working_dir=None)),
 ('prefittedsoftvotingclassifier',
  PreFittedSoftVotingClassifier(classification_labels=None,
                                estimators=[('12',
                                             Pipeline(memory=None,
                                                      steps=[('standardscalerwrapper',
                                                              <azureml.automl.runtime.shared.model_wrappers.StandardScalerWrapper object at 0x7fcf106e0828>),
                                                             ('lightgbmclassifier',
                                                              LightGBMClassifier(

In [16]:
from pprint import pprint

def print_model(model, prefix=""):
    for step in model.steps:
        print(prefix + step[0])
        if hasattr(step[1], 'estimators') and hasattr(step[1], 'weights'):
            pprint({'estimators': list(
                e[0] for e in step[1].estimators), 'weights': step[1].weights})
            print()
            for estimator in step[1].estimators:
                print_model(estimator[1], estimator[0] + ' - ')
        else:
            pprint(step[1].get_params())
            print()

print_model(fitted_model_automl)

datatransformer
{'enable_dnn': None,
 'enable_feature_sweeping': None,
 'feature_sweeping_config': None,
 'feature_sweeping_timeout': None,
 'featurization_config': None,
 'force_text_dnn': None,
 'is_cross_validation': None,
 'is_onnx_compatible': None,
 'logger': None,
 'observer': None,
 'task': None,
 'working_dir': None}

prefittedsoftvotingclassifier
{'estimators': ['12', '21', '0', '18', '26', '11', '32', '3', '36', '14'],
 'weights': [0.07692307692307693,
             0.23076923076923078,
             0.07692307692307693,
             0.07692307692307693,
             0.15384615384615385,
             0.07692307692307693,
             0.07692307692307693,
             0.07692307692307693,
             0.07692307692307693,
             0.07692307692307693]}

12 - standardscalerwrapper
{'class_name': 'StandardScaler',
 'copy': True,
 'module_name': 'sklearn.preprocessing._data',
 'with_mean': False,
 'with_std': True}

12 - lightgbmclassifier
{'boosting_type': 'gbdt',
 'class_wei

In [17]:
#Save the best model - download pkl and register model in Azure
model_name = 'best_model'
model = best_run_automl.register_model(model_name, model_path='outputs/model.pkl')

best_run_automl.download_file('outputs/model.pkl', 'automl_output/model.pkl')
best_run_automl.download_file('outputs/scoring_file_v_1_0_0.py', 'automl_output/score.py')
print(model)

Model(workspace=Workspace.create(name='quick-starts-ws-143180', subscription_id='510b94ba-e453-4417-988b-fbdc37b55ca7', resource_group='aml-quickstarts-143180'), name=best_model, id=best_model:1, version=1, tags={}, properties={})


## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [18]:
service_name = 'capstone-service'
inference_config = InferenceConfig(entry_script='automl_output/score.py')
service = Model.deploy(ws, service_name, [model], inference_config, overwrite=True)
service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-04-19 20:35:55+00:00 Creating Container Registry if not exists..
2021-04-19 20:36:11+00:00 Use the existing image.
2021-04-19 20:36:12+00:00 Generating deployment configuration.
2021-04-19 20:36:14+00:00 Submitting deployment to compute..
2021-04-19 20:36:45+00:00 Checking the status of deployment capstone-service..
2021-04-19 20:41:44+00:00 Checking the status of inference endpoint capstone-service.
Succeeded
ACI service creation operation finished, operation "Succeeded"


TODO: In the cell below, send a request to the web service you deployed to test it.

In [24]:
print(service.state)

# service.update(enable_app_insights=True)

scoring_uri = service.scoring_uri

print(f'\nservice state: {service.state}\n')
print(f'scoring URI: \n{service.scoring_uri}\n')
print(f'swagger URI: \n{service.swagger_uri}\n')

print(service.scoring_uri)
print(service.swagger_uri)

import requests
import json

data = {"data":
        [
          {
            "PassengerId": 812,
            "Pclass": 2,
            "Name": "Berriman, Mr. William John",
            "Sex": "male",
            "Age": 23.0,
            "SibSp": 0,
            "Parch": 0, 
            "Ticket": "28425",
            "Fare": 13.0,
            "Cabin": "",
            "Embarked": "S",
          },
          {
            "PassengerId": 813,
            "Pclass": 1,
            "Name": "Lesurer, Mr. Gustave J",
            "Sex": "male",
            "Age": 35.0,
            "SibSp": 0,
            "Parch": 0, 
            "Ticket": "PC 17755",
            "Fare": 512.3292,
            "Cabin": "B101",
            "Embarked": "S",

          }
      ]
    }

input_data=json.dumps(data)
# with open("data.json","w") as file:
#     file.write(input_data)

# headers={"Content-Type":"application/json"}

# resp=requests.post(scoring_uri, input_data, headers=headers)
# print("Response is: ",resp.json())

Healthy

service state: Healthy

scoring URI: 
http://b11480c0-5026-474d-b44c-1a52fbb3fb48.southcentralus.azurecontainer.io/score

swagger URI: 
http://b11480c0-5026-474d-b44c-1a52fbb3fb48.southcentralus.azurecontainer.io/swagger.json

http://b11480c0-5026-474d-b44c-1a52fbb3fb48.southcentralus.azurecontainer.io/score
http://b11480c0-5026-474d-b44c-1a52fbb3fb48.southcentralus.azurecontainer.io/swagger.json


In [25]:
output = service.run(input_data)
print(output)

{"result": [0, 1]}


In [27]:
service.delete()

TODO: In the cell below, print the logs of the web service and delete the service

In [26]:
logs = service.get_logs()
for line in logs.split('\n'):
    print(line)

2021-04-19T20:48:55,011073300+00:00 - gunicorn/run 
2021-04-19T20:48:55,038908600+00:00 - iot-server/run 
2021-04-19T20:48:55,012057700+00:00 - nginx/run 
/usr/sbin/nginx: /azureml-envs/azureml_1c91e9a5baadb44a18d64bdd7f599d89/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_1c91e9a5baadb44a18d64bdd7f599d89/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_1c91e9a5baadb44a18d64bdd7f599d89/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_1c91e9a5baadb44a18d64bdd7f599d89/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_1c91e9a5baadb44a18d64bdd7f599d89/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
2021-04-19T20:48:55,220025400+00:00 - rsyslog/run 
rsyslogd